In [322]:
import logging
from itertools import combinations

import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

from icecream import ic

import math


In [ ]:
import math
import logging
import random
from itertools import combinations
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

class Problem:
    _graph: nx.Graph
    _alpha: float
    _beta: float

    def __init__(
        self,
        num_cities: int = 30,
        *,
        alpha: float = 1.0,
        beta: float = 1.0,
        density: float = 0.5,
        seed: int = 42,
    ):
        rng = np.random.default_rng(seed)
        self._alpha = alpha
        self._beta = beta
        cities = rng.random(size=(num_cities, 2))
        cities[0, 0] = cities[0, 1] = 0.5

        self._graph = nx.Graph()
        self._graph.add_node(0, pos=(cities[0, 0], cities[0, 1]), gold=0)
        for c in range(1, num_cities):
            self._graph.add_node(c, pos=(cities[c, 0], cities[c, 1]), gold=(1 + 999 * rng.random()))

        tmp = cities[:, np.newaxis, :] - cities[np.newaxis, :, :]
        d = np.sqrt(np.sum(np.square(tmp), axis=-1))
        for c1, c2 in combinations(range(num_cities), 2):
            if rng.random() < density or c2 == c1 + 1:
                self._graph.add_edge(c1, c2, dist=d[c1, c2])

        assert nx.is_connected(self._graph)
  
        self._shortest_paths_lens = dict(nx.all_pairs_dijkstra_path_length(self._graph, weight='dist'))

        self._paths = dict(nx.all_pairs_dijkstra_path(self._graph, weight='dist'))

        self._edge_dists = {}
        for u, v, data in self._graph.edges(data=True):
            self._edge_dists[(u, v)] = data['dist']
            self._edge_dists[(v, u)] = data['dist'] 

    @property
    def graph(self) -> nx.Graph:
        return nx.Graph(self._graph)

    @property
    def alpha(self):
        return self._alpha

    @property
    def beta(self):
        return self._beta

    def cost(self, path, weight):
        dist = nx.path_weight(self._graph, path, weight='dist')
        return dist + (self._alpha * dist * weight) ** self._beta

    def baseline(self):
        total_cost = 0
        for dest, path in nx.single_source_dijkstra_path(
            self._graph, source=0, weight='dist'
        ).items():
            cost = 0
            for c1, c2 in zip(path, path[1:]):
                cost += self.cost([c1, c2], 0)
                cost += self.cost([c1, c2], self._graph.nodes[dest]['gold'])
            total_cost += cost
        return total_cost

    def _get_real_path_cost(self, u, v, load):

        if u == v: return 0.0
        path = self._paths[u][v]
        total_cost = 0.0
        
        for i in range(len(path) - 1):
            n1, n2 = path[i], path[i+1]
            d = self._edge_dists[(n1, n2)]
            segment_cost = d + (self._alpha * d * load) ** self._beta
            total_cost += segment_cost
            
        return total_cost

    def split(self, sequence):
        n = len(sequence)
        golds = [self._graph.nodes[node]['gold'] for node in sequence]
        
        V = [float('inf')] * (n + 1)
        V[0] = 0.0
        P = [0] * (n + 1)

        for i in range(n):
            if V[i] == float('inf'): continue
            
            current_load = 0.0
            trip_cost = 0.0
            u = 0 
            
            for j in range(i + 1, n + 1):
                v = sequence[j-1]
                gold_v = golds[j-1]
                
                try:
                    cost_uv = self._get_real_path_cost(u, v, current_load)
                except KeyError:
                    break 

                trip_cost += cost_uv
                
                current_load += gold_v
                
                try:
                    return_cost = self._get_real_path_cost(v, 0, current_load)
                except KeyError:
                    break
                    
                total_trip_cost = trip_cost + return_cost
                
                if V[i] + total_trip_cost < V[j]:
                    V[j] = V[i] + total_trip_cost
                    P[j] = i
                
                u = v

        routes = []
        curr = n
        if V[n] == float('inf'):
            return float('inf'), []

        while curr > 0:
            start = P[curr]
            routes.append([0] + sequence[start:curr] + [0])
            curr = start
            
        routes.reverse()
        return V[n], routes

    def generate_nearest_neighbor_individual(self):
        unvisited = set(range(1, len(self._graph)))
        current_node = 0
        tour = []
        while unvisited:
            next_node = min(unvisited, key=lambda n: self._shortest_paths_lens[current_node][n])
            tour.append(next_node)
            unvisited.remove(next_node)
            current_node = next_node
        return tour

    def generate_sweep_individual(self):
        depot_pos = self._graph.nodes[0]['pos']
        nodes_with_angles = []
        for i in range(1, len(self._graph)):
            pos = self._graph.nodes[i]['pos']
            dy = pos[1] - depot_pos[1]
            dx = pos[0] - depot_pos[0]
            angle = math.atan2(dy, dx)
            nodes_with_angles.append((angle, i))
        nodes_with_angles.sort(key=lambda x: x[0])
        return [n for _, n in nodes_with_angles]
    
    def generate_farthest_distance_individual(self):
        nodes = list(range(1, len(self._graph)))
        nodes.sort(key=lambda x: self._shortest_paths_lens[0][x], reverse=True)
        return nodes
    
    def generate_grasp_individual(self, k=3):
        unvisited = list(range(1, len(self._graph)))
        current_node = 0
        tour = []
        while unvisited:
            unvisited.sort(key=lambda n: self._shortest_paths_lens[current_node][n])
            candidates = unvisited[:k]
            next_node = random.choice(candidates)
            tour.append(next_node)
            unvisited.remove(next_node)
            current_node = next_node
        return tour

    def generate_cost_aware_greedy(self):
        unvisited = set(range(1, len(self._graph)))
        curr = 0
        load = 0
        tour = []
        while unvisited:
            best_n, best_inc = None, float('inf')
            for cand in unvisited:
                dist = self._shortest_paths_lens[curr][cand]
                # Stima costo locale
                step = dist + (self._alpha * dist * load) ** self._beta
                if step < best_inc:
                    best_inc = step; best_n = cand
            tour.append(best_n)
            unvisited.remove(best_n)
            curr = best_n
            load += self._graph.nodes[best_n]['gold']
        return tour

    def generate_clarke_wright_individual(self):
        n = len(self._graph) - 1
        nodes = list(range(1, n + 1))
        routes = {i: [i] for i in nodes}
        node_to_route_id = {i: i for i in nodes}
        savings = []
        golds = {i: self._graph.nodes[i]['gold'] for i in nodes}

        for i in range(1, n + 1):
            for j in range(i + 1, n + 1):
                try:
                    d_0i = self._shortest_paths_lens[0][i]; d_i0 = self._shortest_paths_lens[i][0]
                    d_0j = self._shortest_paths_lens[0][j]; d_j0 = self._shortest_paths_lens[j][0]
                    d_ij = self._shortest_paths_lens[i][j]; d_ji = self._shortest_paths_lens[j][i]
                except KeyError: continue

                g_i = golds[i]; g_j = golds[j]
                
                c_i = d_0i + (d_i0 + (self._alpha * d_i0 * g_i)**self._beta)
                c_j = d_0j + (d_j0 + (self._alpha * d_j0 * g_j)**self._beta)
                base = c_i + c_j
                
                c_ij = d_0i + (d_ij + (self._alpha * d_ij * g_i)**self._beta) + \
                       (d_j0 + (self._alpha * d_j0 * (g_i + g_j))**self._beta)
                c_ji = d_0j + (d_ji + (self._alpha * d_ji * g_j)**self._beta) + \
                       (d_i0 + (self._alpha * d_i0 * (g_j + g_i))**self._beta)
                
                sav = base - min(c_ij, c_ji)
                if sav > 0: savings.append((sav, i, j))
        
        savings.sort(key=lambda x: x[0], reverse=True)
        
        for _, i, j in savings:
            ri = node_to_route_id[i]; rj = node_to_route_id[j]
            if ri != rj:
                route_i = routes[ri]; route_j = routes[rj]
                if route_i[-1] == i and route_j[0] == j: new_r = route_i + route_j
                elif route_j[-1] == j and route_i[0] == i: new_r = route_j + route_i
                elif route_i[-1] == i and route_j[-1] == j: new_r = route_i + route_j[::-1]
                elif route_i[0] == i and route_j[0] == j: new_r = route_i[::-1] + route_j
                else: continue
                del routes[ri]; del routes[rj]
                routes[new_r[0]] = new_r
                for node in new_r: node_to_route_id[node] = new_r[0]
                
        return [n for r in routes.values() for n in r]

    def generate_cheapest_insertion_individual(self):
        n = len(self._graph) - 1
        unvisited = list(range(1, n + 1))
        farthest = max(unvisited, key=lambda x: self._shortest_paths_lens[0][x])
        tour = [0, farthest, 0]
        unvisited.remove(farthest)
        
        while unvisited:
            best_n, best_pos, best_inc = None, -1, float('inf')
            for node in unvisited:
                for i in range(len(tour) - 1):
                    u, v = tour[i], tour[i+1]
                    try:
                        d_uv = self._shortest_paths_lens[u][v]
                        d_un = self._shortest_paths_lens[u][node]
                        d_nv = self._shortest_paths_lens[node][v]
                        g = self._graph.nodes[node]['gold']
                        delta = d_un + d_nv - d_uv
                        inc = delta + (delta * g * self._alpha)**self._beta
                        if inc < best_inc:
                            best_inc = inc; best_n = node; best_pos = i+1
                    except KeyError: continue
            if best_n:
                tour.insert(best_pos, best_n)
                unvisited.remove(best_n)
            else: break
        return tour[1:-1]
    
    def generate_regret_insertion_individual(self):
        return self.generate_cheapest_insertion_individual() 
        
    def generate_gold_sorted_individual(self):
        nodes = list(range(1, len(self._graph)))
        nodes.sort(key=lambda x: (self._graph.nodes[x]['gold'], -self._shortest_paths_lens[0][x]), reverse=True)
        return nodes
    
    def fast_2opt_safe(self, route, max_iter=500):
        best_route = route[:]

        def route_dist(r):
            d = 0
            for i in range(len(r)-1):
                d += self._shortest_paths_lens[r[i]][r[i+1]]
            return d

        best_dist = route_dist([0] + best_route + [0]) 
        
        improved = True
        it = 0
        while improved and it < max_iter:
            improved = False
            it += 1
            for i in range(len(best_route) - 1):
                for j in range(i + 1, len(best_route)):
                    if j - i == 1: continue 
                    
                    new_route = best_route[:]
                    new_route[i:j] = best_route[i:j][::-1]
                    
                    new_dist = route_dist([0] + new_route + [0])
                    
                    if new_dist < best_dist - 1e-6: 
                        best_route = new_route
                        best_dist = new_dist
                        improved = True
                        break 
                if improved: break
        return best_route

    def generate_closest_distance_individual(self):
        nodes = list(range(1, len(self._graph)))
        nodes.sort(key=lambda x: self._shortest_paths_lens[0][x])
        return nodes
    
    def generate_randomized_clarke_wright_individual(self, top_k=5):
        n = len(self._graph) - 1
        nodes = list(range(1, n + 1))
        routes = {i: [i] for i in nodes}
        node_to_route_id = {i: i for i in nodes}
        savings = []
        golds = {i: self._graph.nodes[i]['gold'] for i in nodes}

        for i in range(1, n + 1):
            for j in range(i + 1, n + 1):
                try:
                    d_0i = self._shortest_paths_lens[0][i]; d_i0 = self._shortest_paths_lens[i][0]
                    d_0j = self._shortest_paths_lens[0][j]; d_j0 = self._shortest_paths_lens[j][0]
                    d_ij = self._shortest_paths_lens[i][j]; d_ji = self._shortest_paths_lens[j][i]
                except KeyError: continue

                g_i = golds[i]; g_j = golds[j]
                
                base = (d_0i + (d_i0 + (self._alpha * d_i0 * g_i)**self._beta)) + \
                       (d_0j + (d_j0 + (self._alpha * d_j0 * g_j)**self._beta))
                
                c_ij = d_0i + (d_ij + (self._alpha * d_ij * g_i)**self._beta) + \
                       (d_j0 + (self._alpha * d_j0 * (g_i + g_j))**self._beta)
                c_ji = d_0j + (d_ji + (self._alpha * d_ji * g_j)**self._beta) + \
                       (d_i0 + (self._alpha * d_i0 * (g_j + g_i))**self._beta)
                
                sav = base - min(c_ij, c_ji)
                if sav > 0: savings.append((sav, i, j))
        
        savings.sort(key=lambda x: x[0], reverse=True)
        
        while savings:
            candidates = savings[:top_k]
            chosen_idx = random.randint(0, len(candidates) - 1)
            sav, i, j = savings.pop(chosen_idx) 

            ri = node_to_route_id[i]; rj = node_to_route_id[j]
            if ri != rj:
                route_i = routes[ri]; route_j = routes[rj]
                
                if route_i[-1] == i and route_j[0] == j: new_r = route_i + route_j
                elif route_j[-1] == j and route_i[0] == i: new_r = route_j + route_i
                elif route_i[-1] == i and route_j[-1] == j: new_r = route_i + route_j[::-1]
                elif route_i[0] == i and route_j[0] == j: new_r = route_i[::-1] + route_j
                else: continue
                
                del routes[ri]; del routes[rj]
                routes[new_r[0]] = new_r
                for node in new_r: node_to_route_id[node] = new_r[0]
                
        return [n for r in routes.values() for n in r]

In [ ]:
import random

num_cities = 100 
problem = Problem(num_cities, alpha=1.0, beta=1, density=0.5) 
#cromosoma = list(range(1, num_cities)) 
#random.shuffle(cromosoma) 

#cromosoma = problem.generate_nearest_neighbor_individual()
#cromosoma = problem.generate_sweep_individual()
cromosoma = problem.generate_farthest_distance_individual()

print(f"Cromosoma (Sequence): {cromosoma}")
costo_totale, viaggi_reali = problem.split(cromosoma)

baseline_cost = problem.baseline()

print(f"Fitness (Costo): {costo_totale:.4f}")
print(f"Viaggi Decodificati: {viaggi_reali}")

print(f"Costo baseline: {baseline_cost}")

improvement = ((baseline_cost-costo_totale)/baseline_cost)*100

print(f"Miglioramento percentuale: {improvement}%")


Cromosoma (Sequence): [2, 42, 11, 49, 87, 25, 54, 80, 90, 65, 81, 29, 15, 51, 69, 75, 84, 95, 96, 86, 37, 88, 62, 68, 57, 50, 67, 52, 74, 34, 71, 97, 89, 5, 22, 83, 41, 13, 36, 8, 48, 77, 26, 61, 93, 58, 12, 1, 64, 45, 24, 4, 63, 14, 98, 3, 9, 27, 18, 76, 39, 92, 99, 94, 6, 79, 44, 20, 35, 19, 17, 55, 10, 21, 7, 78, 31, 46, 38, 47, 43, 72, 23, 53, 16, 85, 56, 33, 30, 91, 40, 66, 32, 60, 73, 59, 82, 28, 70]
Fitness (Costo): 18811.2898
Viaggi Decodificati: [[0, 2, 0], [0, 42, 0], [0, 11, 0], [0, 49, 0], [0, 87, 0], [0, 25, 0], [0, 54, 0], [0, 80, 0], [0, 90, 0], [0, 65, 0], [0, 81, 0], [0, 29, 0], [0, 15, 0], [0, 51, 0], [0, 69, 0], [0, 75, 0], [0, 84, 0], [0, 95, 0], [0, 96, 0], [0, 86, 0], [0, 37, 0], [0, 88, 0], [0, 62, 0], [0, 68, 0], [0, 57, 0], [0, 50, 0], [0, 67, 0], [0, 52, 0], [0, 74, 0], [0, 34, 0], [0, 71, 0], [0, 97, 0], [0, 89, 0], [0, 5, 0], [0, 22, 0], [0, 83, 0], [0, 41, 0], [0, 13, 0], [0, 36, 0], [0, 8, 0], [0, 48, 0], [0, 77, 0], [0, 26, 0], [0, 61, 0], [0, 93, 0], [0,

In [ ]:
import random
import networkx as nx
import numpy as np

def run_extended_benchmark(num_cities=50):
    print(f"=== AVVIO BENCHMARK ESTESO SU {num_cities} CITTÀ ===\n")

    scenarios = [
        {
            "name": "1. Standard",
            "alpha": 1.0, "beta": 1.0, "density": 0.5,
            "desc": "Equilibrio base (Lineare)."
        },
        {
            "name": "2. Carico Pesante (Beta Alto)",
            "alpha": 1.0, "beta": 2.0, "density": 0.5,
            "desc": "Il peso esplode (Quadratico). Meglio viaggi brevi."
        },
        {
            "name": "3. Labirinto (Grafo Sparso)",
            "alpha": 1.0, "beta": 1.0, "density": 0.2,
            "desc": "Poche strade dirette. Geometria difficile."
        },
        {
            "name": "4. Autostrade Ovunque (Grafo Completo)",
            "alpha": 1.0, "beta": 1.0, "density": 1.0,
            "desc": "Tutti collegati con tutti. Geometria euclidea pura."
        },
        {
            "name": "5. Economia di Scala (Beta Basso)",
            "alpha": 1.0, "beta": 0.5, "density": 0.5, 
            "desc": "Il peso costa poco (Radice). Incentiva viaggi lunghissimi."
        }
    ]

    for sc in scenarios:
        print(f"\n--- {sc['name']} ---")
        print(f"Params: alpha={sc['alpha']}, beta={sc['beta']}, density={sc['density']}")
        print(f"Desc: {sc['desc']}")
        
        try:
            p = Problem(
                num_cities, 
                alpha=sc['alpha'], 
                beta=sc['beta'], 
                density=sc['density'],
                seed=42 
            )
        except AssertionError:
            print("Skipped: Grafo non connesso (prova density più alta o altro seed).")
            continue
        except Exception as e:
            print(f"Errore generazione grafo: {e}")
            continue

        try:
            base_cost = p.baseline()
        except nx.NetworkXNoPath:
             print("Errore Baseline: Grafo sconnesso durante il calcolo.")
             continue

        print(f"{'METODO':<25} | {'COSTO':<12} | {'VIAGGI':<8} | {'MIGLIORAMENTO':<15}")
        print("-" * 75)
        print(f"{'Baseline':<25} | {base_cost:<12.2f} | {'N/A':<8} | {'0.00%':<15}")

        strategies = [
            ("Nearest Neighbor", p.generate_nearest_neighbor_individual),
            ("Sweep Algorithm", p.generate_sweep_individual),
            ("Farthest Distance", p.generate_farthest_distance_individual),
            ("Random Shuffle", None)
        ]

        results = []

        for name, func in strategies:
            if name == "Random Shuffle":
                ind = list(range(1, num_cities))
                random.seed(10)
                random.shuffle(ind)
            else:
                ind = func()

            cost, routes = p.split(ind)
            
            improvement = ((base_cost - cost) / base_cost) * 100
            num_viaggi = len(routes)
            results.append((name, cost, num_viaggi, improvement))

        results.sort(key=lambda x: x[1])

        for name, cost, n_trips, imp in results:
            print(f"{name:<25} | {cost:<12.2f} | {n_trips:<8} | {imp:<6.2f}%")
        
        print(f"-> VINCITORE: {results[0][0]}")

run_extended_benchmark(num_cities=50)

=== AVVIO BENCHMARK ESTESO SU 50 CITTÀ ===


--- 1. Standard ---
Params: alpha=1.0, beta=1.0, density=0.5
Desc: Equilibrio base (Lineare).
METODO                    | COSTO        | VIAGGI   | MIGLIORAMENTO  
---------------------------------------------------------------------------
Baseline                  | 9408.58      | N/A      | 0.00%          
Sweep Algorithm           | 9405.00      | 42       | 0.04  %
Nearest Neighbor          | 9405.33      | 44       | 0.03  %
Random Shuffle            | 9408.42      | 48       | 0.00  %
Farthest Distance         | 9408.58      | 49       | -0.00 %
-> VINCITORE: Sweep Algorithm

--- 2. Carico Pesante (Beta Alto) ---
Params: alpha=1.0, beta=2.0, density=0.5
Desc: Il peso esplode (Quadratico). Meglio viaggi brevi.
METODO                    | COSTO        | VIAGGI   | MIGLIORAMENTO  
---------------------------------------------------------------------------
Baseline                  | 1876144.04   | N/A      | 0.00%          
Nearest Neighb

In [ ]:
def smart_population_initialization(problem, pop_size):
    """
    Inizializza la popolazione adattandosi ai parametri fisici del problema (beta).
    """
    population = []

    use_heuristics = (problem.beta < 1.0)
    
    print(f"Logica Inizializzazione: {'Euristiche Attive' if use_heuristics else 'Focus Random'}")

    if problem.beta < 1.5:
        population.append(problem.generate_nearest_neighbor_individual())
    if use_heuristics:
        population.append(problem.generate_sweep_individual())
        population.append(problem.generate_farthest_distance_individual())

    while len(population) < pop_size:
        nodes = list(range(1, len(problem._graph)))
        random.shuffle(nodes)
        population.append(nodes)
        
    return population

In [ ]:
def test_smart_init_performance(num_cities=50, pop_size=50):
    print(f"=== TEST SMART INITIALIZATION ({num_cities} Città, Pop: {pop_size}) ===\n")

    scenarios = [
        {"name": "Scenario A: Standard", "alpha": 1.0, "beta": 1.0, "density": 0.5},
        {"name": "Scenario B: Peso Punitivo", "alpha": 1.0, "beta": 2.0, "density": 0.5},
        {"name": "Scenario C: Economy (Green)", "alpha": 1.0, "beta": 0.5, "density": 0.5},
    ]

    for sc in scenarios:
        print(f"\n--- {sc['name']} (Beta={sc['beta']}) ---")
      
        try:
            p = Problem(num_cities, alpha=sc['alpha'], beta=sc['beta'], density=sc['density'], seed=42)
        except AssertionError:
            print("Skipped (Grafo non connesso)")
            continue

        baseline = p.baseline()

        population = smart_population_initialization(p, pop_size)
      
        best_cost = float('inf')
        best_index = -1
        best_routes = []
        costs = []

        for i, ind in enumerate(population):
            cost, routes = p.split(ind)
            costs.append(cost)
            
            if cost < best_cost:
                best_cost = cost
                best_index = i
                best_routes = routes

        winner_type = "Random" 
        
        if p.beta < 1.0:
            if best_index == 0: winner_type = "Nearest Neighbor"
            elif best_index == 1: winner_type = "Sweep Algorithm"
            elif best_index == 2: winner_type = "Farthest Distance"
            else: winner_type = f"Random (Indice {best_index})"
            
        elif p.beta < 1.5: 
            if best_index == 0: winner_type = "Nearest Neighbor"
            else: winner_type = f"Random (Indice {best_index})"
            
        else: 
             winner_type = f"Random (Indice {best_index})"

        avg_cost = sum(costs) / len(costs)
        improvement = ((baseline - best_cost) / baseline) * 100
        
        print(f"{'METRICA':<25} | {'VALORE'}")
        print("-" * 50)
        print(f"{'Costo Baseline':<25} | {baseline:.2f}")
        print(f"{'Migliore Smart Pop':<25} | {best_cost:.2f}")
        print(f"{'Fonte Vincitore':<25} | {winner_type}")
        print(f"{'Miglioramento':<25} | {improvement:.2f}%")
        print(f"{'Media Popolazione':<25} | {avg_cost:.2f}")
        print(f"{'Numero Viaggi':<25} | {len(best_routes)}")

test_smart_init_performance(num_cities=100, pop_size=200)

=== TEST SMART INITIALIZATION (100 Città, Pop: 200) ===


--- Scenario A: Standard (Beta=1.0) ---
Logica Inizializzazione: Focus Random
METRICA                   | VALORE
--------------------------------------------------
Costo Baseline            | 18811.32
Migliore Smart Pop        | 18807.06
Fonte Vincitore           | Nearest Neighbor
Miglioramento             | 0.02%
Media Popolazione         | 18810.97
Numero Viaggi             | 92

--- Scenario B: Peso Punitivo (Beta=2.0) ---
Logica Inizializzazione: Focus Random
METRICA                   | VALORE
--------------------------------------------------
Costo Baseline            | 4696149.78
Migliore Smart Pop        | 4356534.81
Fonte Vincitore           | Random (Indice 124)
Miglioramento             | 7.23%
Media Popolazione         | 4583579.11
Numero Viaggi             | 92

--- Scenario C: Economy (Green) (Beta=0.5) ---
Logica Inizializzazione: Euristiche Attive
METRICA                   | VALORE
-------------------------------

In [ ]:
def super_smart_initialization(problem, pop_size):
    population = []

    population.append(problem.generate_nearest_neighbor_individual())
    
    population.append(problem.generate_cost_aware_greedy())
    
    if problem.beta < 1.5:
        population.append(problem.generate_sweep_individual())
        population.append(problem.generate_farthest_distance_individual())

    grasp_quota = int(pop_size * 0.5)
    
    k_grasp = 3 if problem.beta < 1.0 else 6
    
    for _ in range(grasp_quota):
        population.append(problem.generate_grasp_individual(k=k_grasp))
        
    while len(population) < pop_size:
        nodes = list(range(1, len(problem._graph)))
        random.shuffle(nodes)
        population.append(nodes)
        
    return population

In [ ]:
def test_super_smart_init_performance(num_cities=100, pop_size=200):
    print(f"=== TEST SUPER SMART INITIALIZATION ({num_cities} Città, Pop: {pop_size}) ===\n")

    scenarios = [
        {"name": "Scenario A: Standard", "alpha": 1.0, "beta": 1.0, "density": 0.5},
        {"name": "Scenario B: Peso Punitivo", "alpha": 1.0, "beta": 2.0, "density": 0.5},
        {"name": "Scenario C: Economy (Green)", "alpha": 1.0, "beta": 0.5, "density": 0.5},
    ]

    for sc in scenarios:
        print(f"\n--- {sc['name']} (Beta={sc['beta']}) ---")
        
        # Setup
        try:
            p = Problem(num_cities, alpha=sc['alpha'], beta=sc['beta'], density=sc['density'], seed=42)
        except AssertionError:
            print("Skipped (Grafo non connesso)")
            continue

        baseline = p.baseline()

        population = super_smart_initialization(p, pop_size)
        
        best_cost = float('inf')
        best_index = -1
        best_routes = []
        costs = []

        for i, ind in enumerate(population):
            cost, routes = p.split(ind)
            costs.append(cost)
            
            if cost < best_cost:
                best_cost = cost
                best_index = i
                best_routes = routes
        
        winner_type = "Unknown"
        grasp_quota = int(pop_size * 0.5) 
        
        if p.beta < 1.5:
            grasp_start_index = 4
            
            if best_index == 0: winner_type = "Nearest Neighbor (Classic)"
            elif best_index == 1: winner_type = "Cost-Aware Greedy (New!)"
            elif best_index == 2: winner_type = "Sweep Algorithm"
            elif best_index == 3: winner_type = "Farthest Distance"
            elif grasp_start_index <= best_index < (grasp_start_index + grasp_quota):
                k_val = 3 if p.beta < 1.0 else 6
                winner_type = f"GRASP (k={k_val}, Indice {best_index})"
            else:
                winner_type = f"Random Shuffle (Indice {best_index})"

        else:
            grasp_start_index = 2
            
            if best_index == 0: winner_type = "Nearest Neighbor (Classic)"
            elif best_index == 1: winner_type = "Cost-Aware Greedy (New!)"
            elif grasp_start_index <= best_index < (grasp_start_index + grasp_quota):
                winner_type = f"GRASP (k=6, Indice {best_index})"
            else:
                winner_type = f"Random Shuffle (Indice {best_index})"

        avg_cost = sum(costs) / len(costs)
        improvement = ((baseline - best_cost) / baseline) * 100
        
        print(f"{'METRICA':<30} | {'VALORE'}")
        print("-" * 55)
        print(f"{'Costo Baseline':<30} | {baseline:.2f}")
        print(f"{'Migliore Smart Pop':<30} | {best_cost:.2f}")
        print(f"{'Fonte Vincitore':<30} | {winner_type}")
        print(f"{'Miglioramento':<30} | {improvement:.2f}%")
        print(f"{'Media Popolazione':<30} | {avg_cost:.2f}")
        print(f"{'Numero Viaggi':<30} | {len(best_routes)}")

test_super_smart_init_performance(num_cities=100, pop_size=200)

=== TEST SUPER SMART INITIALIZATION (100 Città, Pop: 200) ===


--- Scenario A: Standard (Beta=1.0) ---
METRICA                        | VALORE
-------------------------------------------------------
Costo Baseline                 | 18811.32
Migliore Smart Pop             | 18804.26
Fonte Vincitore                | Sweep Algorithm
Miglioramento                  | 0.04%
Media Popolazione              | 18810.36
Numero Viaggi                  | 86

--- Scenario B: Peso Punitivo (Beta=2.0) ---
METRICA                        | VALORE
-------------------------------------------------------
Costo Baseline                 | 4696149.78
Migliore Smart Pop             | 4304286.48
Fonte Vincitore                | GRASP (k=6, Indice 85)
Miglioramento                  | 8.34%
Media Popolazione              | 4548112.21
Numero Viaggi                  | 87

--- Scenario C: Economy (Green) (Beta=0.5) ---
METRICA                        | VALORE
-------------------------------------------------------
C

In [ ]:
def super_smart_initialization_v2(problem, pop_size):
    population = []
    
    print(f"   [Logic V2] Beta={problem.beta} -> Strategia: ", end="")

    try:
        population.append(problem.generate_clarke_wright_individual())
    except Exception as e:
        print(f"C&W Error: {e}") 
        population.append(problem.generate_nearest_neighbor_individual())

    population.append(problem.generate_nearest_neighbor_individual())
    
    if problem.beta >= 1.5:
        print("CLARKE & WRIGHT + GRASP (Peso Punitivo)")
        population.append(problem.generate_cost_aware_greedy())
        
        grasp_quota = int(pop_size * 0.4)
        for _ in range(grasp_quota):
             population.append(problem.generate_grasp_individual(k=6))
             
    else:
        print("GEOMETRICA + GRASP (Economy)")
        population.append(problem.generate_sweep_individual())
        population.append(problem.generate_farthest_distance_individual())

        grasp_quota = int(pop_size * 0.4)
        for _ in range(grasp_quota):
             population.append(problem.generate_grasp_individual(k=3))

    while len(population) < pop_size:
        nodes = list(range(1, len(problem._graph)))
        random.shuffle(nodes)
        population.append(nodes)
        
    return population

In [ ]:
def test_super_smart_v2_performance(num_cities=100, pop_size=200):
    print(f"=== TEST SUPER SMART INITIALIZATION V2 ({num_cities} Città, Pop: {pop_size}) ===\n")

    scenarios = [
        {"name": "Scenario A: Standard", "alpha": 1.0, "beta": 1.0, "density": 0.5},
        {"name": "Scenario B: Peso Punitivo", "alpha": 1.0, "beta": 2.0, "density": 0.5},
        {"name": "Scenario C: Economy (Green)", "alpha": 1.0, "beta": 0.5, "density": 0.5},
    ]

    for sc in scenarios:
        print(f"\n--- {sc['name']} (Beta={sc['beta']}) ---")
        
        try:
            p = Problem(num_cities, alpha=sc['alpha'], beta=sc['beta'], density=sc['density'], seed=42)
        except AssertionError:
            print("Skipped (Grafo non connesso)")
            continue

        baseline = p.baseline()
        population = super_smart_initialization_v2(p, pop_size)
        
        best_cost = float('inf')
        best_index = -1
        best_routes = []
        costs = []

        for i, ind in enumerate(population):
            cost, routes = p.split(ind)
            costs.append(cost)
            
            if cost < best_cost:
                best_cost = cost
                best_index = i
                best_routes = routes

        winner_type = "Unknown"
        grasp_quota = int(pop_size * 0.4) 
        
        if p.beta >= 1.5:
            grasp_start = 3
            grasp_end = grasp_start + grasp_quota
            
            if best_index == 0: winner_type = "Clarke & Wright (Savings)"
            elif best_index == 1: winner_type = "Nearest Neighbor"
            elif best_index == 2: winner_type = "Cost-Aware Greedy"
            elif grasp_start <= best_index < grasp_end:
                winner_type = f"GRASP (k=6, Indice {best_index})"
            else:
                winner_type = f"Random Shuffle (Indice {best_index})"

        else:
            grasp_start = 4
            grasp_end = grasp_start + grasp_quota
            
            if best_index == 0: winner_type = "Clarke & Wright (Savings)"
            elif best_index == 1: winner_type = "Nearest Neighbor"
            elif best_index == 2: winner_type = "Sweep Algorithm"
            elif best_index == 3: winner_type = "Farthest Distance"
            elif grasp_start <= best_index < grasp_end:
                winner_type = f"GRASP (k=3, Indice {best_index})"
            else:
                winner_type = f"Random Shuffle (Indice {best_index})"

        avg_cost = sum(costs) / len(costs)
        improvement = ((baseline - best_cost) / baseline) * 100
        
        print(f"{'METRICA':<30} | {'VALORE'}")
        print("-" * 55)
        print(f"{'Costo Baseline':<30} | {baseline:.2f}")
        print(f"{'Migliore Smart V2':<30} | {best_cost:.2f}")
        print(f"{'Fonte Vincitore':<30} | {winner_type}")
        print(f"{'Miglioramento':<30} | {improvement:.2f}%")
        print(f"{'Media Popolazione':<30} | {avg_cost:.2f}")
        print(f"{'Numero Viaggi':<30} | {len(best_routes)}")

test_super_smart_v2_performance(num_cities=100, pop_size=200)

=== TEST SUPER SMART INITIALIZATION V2 (100 Città, Pop: 200) ===


--- Scenario A: Standard (Beta=1.0) ---
   [Logic V2] Beta=1.0 -> Strategia: GEOMETRICA + GRASP (Economy)
METRICA                        | VALORE
-------------------------------------------------------
Costo Baseline                 | 18811.32
Migliore Smart V2              | 18800.71
Fonte Vincitore                | Clarke & Wright (Savings)
Miglioramento                  | 0.06%
Media Popolazione              | 18810.10
Numero Viaggi                  | 77

--- Scenario B: Peso Punitivo (Beta=2.0) ---
   [Logic V2] Beta=2.0 -> Strategia: CLARKE & WRIGHT + GRASP (Peso Punitivo)
METRICA                        | VALORE
-------------------------------------------------------
Costo Baseline                 | 4696149.78
Migliore Smart V2              | 3888666.04
Fonte Vincitore                | Clarke & Wright (Savings)
Miglioramento                  | 17.19%
Media Popolazione              | 4539115.56
Numero Viaggi        

In [ ]:
def super_smart_initialization_v3(problem, pop_size):
    population = []
    
    print(f"   [Logic V3] Beta={problem.beta} -> ", end="")

    try:
        population.append(problem.generate_clarke_wright_individual())
    except: pass
    try:
        population.append(problem.generate_cheapest_insertion_individual())
    except Exception as e:
        print(f"Insertion Error: {e}")

    population.append(problem.generate_nearest_neighbor_individual())
    
    if problem.beta >= 1.5:
        print("STRATEGIA: C&W + INSERTION + GRASP (Anti-Peso)")
        population.append(problem.generate_cost_aware_greedy())
        
        grasp_quota = int(pop_size * 0.4)
        for _ in range(grasp_quota):
             population.append(problem.generate_grasp_individual(k=6))
             
    else:
        print("STRATEGIA: C&W + GEOMETRICHE + GRASP (Standard/Economy)")
        population.append(problem.generate_sweep_individual())
        population.append(problem.generate_farthest_distance_individual())
        
        grasp_quota = int(pop_size * 0.4)
        for _ in range(grasp_quota):
             population.append(problem.generate_grasp_individual(k=3))

    while len(population) < pop_size:
        nodes = list(range(1, len(problem._graph)))
        random.shuffle(nodes)
        population.append(nodes)
        
    return population

In [ ]:
def test_super_smart_v3_performance(num_cities=100, pop_size=200):
    print(f"=== TEST SUPER SMART INITIALIZATION V3 ({num_cities} Città, Pop: {pop_size}) ===\n")

    scenarios = [
        {"name": "Scenario A: Standard", "alpha": 1.0, "beta": 1.0, "density": 0.5},
        {"name": "Scenario B: Peso Punitivo", "alpha": 1.0, "beta": 2.0, "density": 0.5},
        {"name": "Scenario C: Economy (Green)", "alpha": 1.0, "beta": 0.5, "density": 0.5},
    ]

    for sc in scenarios:
        print(f"\n--- {sc['name']} (Beta={sc['beta']}) ---")
        try:
            p = Problem(num_cities, alpha=sc['alpha'], beta=sc['beta'], density=sc['density'], seed=42)
        except AssertionError:
            print("Skipped (Grafo non connesso)")
            continue

        try:
            baseline = p.baseline()
        except Exception as e:
            print(f"Errore Baseline: {e}")
            continue
        
        population = super_smart_initialization_v3(p, pop_size)
        best_cost = float('inf')
        best_index = -1
        best_routes = []
        costs = []

        for i, ind in enumerate(population):
            cost, routes = p.split(ind)
            costs.append(cost)
            
            if cost < best_cost:
                best_cost = cost
                best_index = i
                best_routes = routes

        winner_type = "Unknown"
        grasp_quota = int(pop_size * 0.4)
        
        if p.beta >= 1.5:
            grasp_start = 4
            grasp_end = grasp_start + grasp_quota
            
            if best_index == 0: winner_type = "Clarke & Wright (Savings)"
            elif best_index == 1: winner_type = "**CHEAPEST INSERTION**"
            elif best_index == 2: winner_type = "Nearest Neighbor"
            elif best_index == 3: winner_type = "Cost-Aware Greedy"
            elif grasp_start <= best_index < grasp_end:
                winner_type = f"GRASP (k=6, Idx {best_index})"
            else:
                winner_type = f"Random Shuffle (Idx {best_index})"

        else:
            grasp_start = 5
            grasp_end = grasp_start + grasp_quota
            
            if best_index == 0: winner_type = "Clarke & Wright (Savings)"
            elif best_index == 1: winner_type = "Cheapest Insertion"
            elif best_index == 2: winner_type = "Nearest Neighbor"
            elif best_index == 3: winner_type = "Sweep Algorithm"
            elif best_index == 4: winner_type = "Farthest Distance"
            elif grasp_start <= best_index < grasp_end:
                winner_type = f"GRASP (k=3, Idx {best_index})"
            else:
                winner_type = f"Random Shuffle (Idx {best_index})"

        avg_cost = sum(costs) / len(costs)
        improvement = ((baseline - best_cost) / baseline) * 100
        
        print(f"{'METRICA':<30} | {'VALORE'}")
        print("-" * 55)
        print(f"{'Costo Baseline':<30} | {baseline:.2f}")
        print(f"{'Migliore Smart V3':<30} | {best_cost:.2f}")
        print(f"{'Fonte Vincitore':<30} | {winner_type}")
        print(f"{'Miglioramento':<30} | {improvement:.2f}%")
        print(f"{'Media Popolazione':<30} | {avg_cost:.2f}")
        print(f"{'Numero Viaggi':<30} | {len(best_routes)}")

test_super_smart_v3_performance(num_cities=100, pop_size=200)

=== TEST SUPER SMART INITIALIZATION V3 (100 Città, Pop: 200) ===


--- Scenario A: Standard (Beta=1.0) ---
   [Logic V3] Beta=1.0 -> STRATEGIA: C&W + GEOMETRICHE + GRASP (Standard/Economy)
METRICA                        | VALORE
-------------------------------------------------------
Costo Baseline                 | 18811.32
Migliore Smart V3              | 18800.71
Fonte Vincitore                | Clarke & Wright (Savings)
Miglioramento                  | 0.06%
Media Popolazione              | 18810.14
Numero Viaggi                  | 77

--- Scenario B: Peso Punitivo (Beta=2.0) ---
   [Logic V3] Beta=2.0 -> STRATEGIA: C&W + INSERTION + GRASP (Anti-Peso)
METRICA                        | VALORE
-------------------------------------------------------
Costo Baseline                 | 4696149.78
Migliore Smart V3              | 3888666.04
Fonte Vincitore                | Clarke & Wright (Savings)
Miglioramento                  | 17.19%
Media Popolazione              | 4542362.95
Numero Vi

C:\Users\feder\AppData\Local\Temp\ipykernel_1296\1450443453.py:305: RuntimeWarning: invalid value encountered in scalar power
  inc = delta + (delta * g * self._alpha)**self._beta


METRICA                        | VALORE
-------------------------------------------------------
Costo Baseline                 | 1535.71
Migliore Smart V3              | 1236.58
Fonte Vincitore                | Clarke & Wright (Savings)
Miglioramento                  | 19.48%
Media Popolazione              | 1434.96
Numero Viaggi                  | 43


In [ ]:
import time

def run_comparative_benchmark_extended(num_cities=100, pop_size=100):
    print(f"==========================================================================")
    print(f"   GRAND PRIX DEFINITIVO: BETA & DENSITÀ ({num_cities} Città, Pop: {pop_size})")
    print(f"==========================================================================\n")
    
    scenarios = [
        {"name": "1. Standard (Ref)",   "alpha": 1.0, "beta": 1.0, "density": 0.5},
        {"name": "2. Punitivo (Beta=2)", "alpha": 1.0, "beta": 2.0, "density": 0.5},
        {"name": "3. Economy (Beta=0.5)","alpha": 1.0, "beta": 0.5, "density": 0.5},

        {"name": "4. Denso Standard",    "alpha": 1.0, "beta": 1.0, "density": 1.0},
        {"name": "5. Denso Punitivo",    "alpha": 1.0, "beta": 2.0, "density": 1.0},
        
        {"name": "6. Sparso Standard",   "alpha": 1.0, "beta": 1.0, "density": 0.2},
        {"name": "7. Sparso Punitivo",   "alpha": 1.0, "beta": 2.0, "density": 0.2},
    ]
    
    strategies = [
        ("1. Smart (Base)", smart_population_initialization),
        ("2. Super V1 (CostAware)", super_smart_initialization),
        ("3. Super V2 (C&W)", super_smart_initialization_v2),
        ("4. Super V3 (Insertion)", super_smart_initialization_v3)
    ]

    for sc in scenarios:
        print(f"\n---> SCENARIO: {sc['name']}")
        print(f"     Params: alpha={sc['alpha']}, beta={sc['beta']}, density={sc['density']}")
        
        try:
            p = Problem(num_cities, alpha=sc['alpha'], beta=sc['beta'], density=sc['density'], seed=42)
            baseline = p.baseline()
        except AssertionError:
            print("     [!] Errore: Grafo non connesso (Seed sfortunato per densità bassa).")
            continue
        except Exception as e:
            print(f"     [!] Errore Baseline o Grafo: {e}")
            continue
            
        print(f"\n{'STRATEGIA':<25} | {'MIGLIOR COSTO':<15} | {'GAP BASELINE':<15} | {'TEMPO (s)':<10}")
        print("-" * 75)
        print(f"{'Baseline (Ref)':<25} | {baseline:<15.2f} | {'0.00%':<15} | {'-':<10}")
        
        for strat_name, strat_func in strategies:
            start_time = time.time()
            
            try:
                population = strat_func(p, pop_size)
                
                best_cost = float('inf')
                for ind in population:
                    c, _ = p.split(ind)
                    if c < best_cost:
                        best_cost = c
                
                elapsed = time.time() - start_time
                improvement = ((baseline - best_cost) / baseline) * 100
                gap_str = f"{improvement:+.2f}%"
                
                print(f"{strat_name:<25} | {best_cost:<15.2f} | {gap_str:<15} | {elapsed:<10.4f}")
                
            except Exception as e:
                print(f"{strat_name:<25} | {'ERRORE':<15} | {'N/A':<15} | {0.0:<10.4f}")

run_comparative_benchmark_extended(num_cities=100, pop_size=100)

   GRAND PRIX DEFINITIVO: BETA & DENSITÀ (100 Città, Pop: 100)


---> SCENARIO: 1. Standard (Ref)
     Params: alpha=1.0, beta=1.0, density=0.5

STRATEGIA                 | MIGLIOR COSTO   | GAP BASELINE    | TEMPO (s) 
---------------------------------------------------------------------------
Baseline (Ref)            | 18811.32        | 0.00%           | -         
Logica Inizializzazione: Focus Random
1. Smart (Base)           | 18807.06        | +0.02%          | 0.8045    
2. Super V1 (CostAware)   | 18804.26        | +0.04%          | 0.7982    
   [Logic V2] Beta=1.0 -> Strategia: GEOMETRICA + GRASP (Economy)
3. Super V2 (C&W)         | 18800.71        | +0.06%          | 0.8391    
   [Logic V3] Beta=1.0 -> STRATEGIA: C&W + GEOMETRICHE + GRASP (Standard/Economy)
4. Super V3 (Insertion)   | 18800.71        | +0.06%          | 0.9233    

---> SCENARIO: 2. Punitivo (Beta=2)
     Params: alpha=1.0, beta=2.0, density=0.5

STRATEGIA                 | MIGLIOR COSTO   | GAP BASELINE 

C:\Users\feder\AppData\Local\Temp\ipykernel_1296\1450443453.py:305: RuntimeWarning: invalid value encountered in scalar power
  inc = delta + (delta * g * self._alpha)**self._beta


4. Super V3 (Insertion)   | 1236.58         | +19.48%         | 0.9272    

---> SCENARIO: 4. Denso Standard
     Params: alpha=1.0, beta=1.0, density=1.0

STRATEGIA                 | MIGLIOR COSTO   | GAP BASELINE    | TEMPO (s) 
---------------------------------------------------------------------------
Baseline (Ref)            | 18266.19        | 0.00%           | -         
Logica Inizializzazione: Focus Random
1. Smart (Base)           | 18264.09        | +0.01%          | 0.6222    
2. Super V1 (CostAware)   | 18259.62        | +0.04%          | 0.6547    
   [Logic V2] Beta=1.0 -> Strategia: GEOMETRICA + GRASP (Economy)
3. Super V2 (C&W)         | 18257.56        | +0.05%          | 0.6642    
   [Logic V3] Beta=1.0 -> STRATEGIA: C&W + GEOMETRICHE + GRASP (Standard/Economy)
4. Super V3 (Insertion)   | 18257.56        | +0.05%          | 0.7533    

---> SCENARIO: 5. Denso Punitivo
     Params: alpha=1.0, beta=2.0, density=1.0

STRATEGIA                 | MIGLIOR COSTO   | GAP B

In [ ]:
def init_v4_ultimate(problem, pop_size):
    pop = []
    
    try: pop.append(problem.generate_clarke_wright_individual())
    except: pass
    
    try: pop.append(problem.generate_regret_insertion_individual()) 
    except: pass
    
    try: pop.append(problem.generate_cheapest_insertion_individual())
    except: pass
    
    pop.append(problem.generate_nearest_neighbor_individual())
    
    if problem.beta >= 1.5:
        pop.append(problem.generate_gold_sorted_individual())
        pop.append(problem.generate_cost_aware_greedy())
        
        grasp_quota = int(pop_size * 0.3)
        for _ in range(grasp_quota): pop.append(problem.generate_grasp_individual(k=6))
        
    else: 
        pop.append(problem.generate_sweep_individual())
        pop.append(problem.generate_farthest_distance_individual())
        
        grasp_quota = int(pop_size * 0.3)
        for _ in range(grasp_quota): pop.append(problem.generate_grasp_individual(k=3))

    while len(pop) < pop_size:
        l = list(range(1, len(problem._graph))); random.shuffle(l); pop.append(l)
        
    return pop

In [ ]:
def run_final_showdown(num_cities=100, pop_size=100):
    print(f"==========================================================================")
    print(f"   GRAND PRIX FINAL: REGRET & GOLD ({num_cities} Città, Pop: {pop_size})")
    print(f"==========================================================================\n")
    
    scenarios = [
        {"name": "1. Standard (Ref)",   "alpha": 1.0, "beta": 1.0, "density": 0.5},
        {"name": "2. Punitivo (Beta=2)", "alpha": 1.0, "beta": 2.0, "density": 0.5},
        {"name": "3. Economy (Beta=0.5)","alpha": 1.0, "beta": 0.5, "density": 0.5},
        {"name": "7. Sparso Punitivo",   "alpha": 1.0, "beta": 2.0, "density": 0.2}, # Il caso critico
    ]
    
    strategies = [
        ("3. Super V2 (C&W)", super_smart_initialization_v2),
        ("4. Super V3 (Insert)", super_smart_initialization_v3),
        ("5. Ultima V4 (Regret)", init_v4_ultimate) # La nuova sfidante
    ]

    for sc in scenarios:
        print(f"\n---> SCENARIO: {sc['name']}")
        
        try:
            p = Problem(num_cities, alpha=sc['alpha'], beta=sc['beta'], density=sc['density'], seed=42)
            baseline = p.baseline()
        except: continue
            
        print(f"\n{'STRATEGIA':<25} | {'MIGLIOR COSTO':<15} | {'GAP BASELINE':<15} | {'TEMPO (s)':<10}")
        print("-" * 75)
        print(f"{'Baseline':<25} | {baseline:<15.2f} | {'0.00%':<15} | {'-':<10}")
        
        for strat_name, strat_func in strategies:
            start_time = time.time()
            try:
                pop = strat_func(p, pop_size)
                best_cost = min([p.split(ind)[0] for ind in pop])
                elapsed = time.time() - start_time
                imp = ((baseline - best_cost) / baseline) * 100
                print(f"{strat_name:<25} | {best_cost:<15.2f} | {imp:+.2f}%{' '*9} | {elapsed:<10.4f}")
            except Exception as e:
                print(f"{strat_name:<25} | ERROR: {e}")

run_final_showdown(num_cities=100, pop_size=100)

   GRAND PRIX FINAL: REGRET & GOLD (100 Città, Pop: 100)


---> SCENARIO: 1. Standard (Ref)

STRATEGIA                 | MIGLIOR COSTO   | GAP BASELINE    | TEMPO (s) 
---------------------------------------------------------------------------
Baseline                  | 18811.32        | 0.00%           | -         
   [Logic V2] Beta=1.0 -> Strategia: GEOMETRICA + GRASP (Economy)
3. Super V2 (C&W)         | 18800.71        | +0.06%          | 0.8203    
   [Logic V3] Beta=1.0 -> STRATEGIA: C&W + GEOMETRICHE + GRASP (Standard/Economy)
4. Super V3 (Insert)      | 18800.71        | +0.06%          | 0.9251    
5. Ultima V4 (Regret)     | 18800.71        | +0.06%          | 1.0177    

---> SCENARIO: 2. Punitivo (Beta=2)

STRATEGIA                 | MIGLIOR COSTO   | GAP BASELINE    | TEMPO (s) 
---------------------------------------------------------------------------
Baseline                  | 4696149.78      | 0.00%           | -         
   [Logic V2] Beta=2.0 -> Strategia: CLARKE 

C:\Users\feder\AppData\Local\Temp\ipykernel_1296\1450443453.py:305: RuntimeWarning: invalid value encountered in scalar power
  inc = delta + (delta * g * self._alpha)**self._beta


4. Super V3 (Insert)      | 1236.58         | +19.48%          | 0.8913    
5. Ultima V4 (Regret)     | 1236.58         | +19.48%          | 1.0037    

---> SCENARIO: 7. Sparso Punitivo

STRATEGIA                 | MIGLIOR COSTO   | GAP BASELINE    | TEMPO (s) 
---------------------------------------------------------------------------
Baseline                  | 5334401.93      | 0.00%           | -         
   [Logic V2] Beta=2.0 -> Strategia: CLARKE & WRIGHT + GRASP (Peso Punitivo)
3. Super V2 (C&W)         | 5177152.82      | +2.95%          | 1.0207    
   [Logic V3] Beta=2.0 -> STRATEGIA: C&W + INSERTION + GRASP (Anti-Peso)
4. Super V3 (Insert)      | 5188072.36      | +2.74%          | 1.1086    
5. Ultima V4 (Regret)     | 5164206.10      | +3.19%          | 1.2406    


In [ ]:
def init_v5_optimized(problem, pop_size):
    pop = []
    raw_individuals = []
    
    raw_individuals.append(problem.generate_closest_distance_individual())
    raw_individuals.append(problem.generate_farthest_distance_individual())
    raw_individuals.append(problem.generate_nearest_neighbor_individual())
    
    try: raw_individuals.append(problem.generate_clarke_wright_individual())
    except: pass
    
    if problem.beta >= 1.5:
        try: raw_individuals.append(problem.generate_cheapest_insertion_individual())
        except: pass
        raw_individuals.append(problem.generate_gold_sorted_individual())
    else:
        raw_individuals.append(problem.generate_sweep_individual())

    for ind in raw_individuals:
        optimized = problem.fast_2opt(ind, max_iter=200) 
        pop.append(optimized)
        
    while len(pop) < pop_size:
        l = list(range(1, len(problem._graph)))
        random.shuffle(l)
        pop.append(l)
        
    return pop

In [ ]:
def run_final_v5_test(num_cities=100, pop_size=100):
    print(f"=== TEST DEFINITIVO: V4 vs V5 (Optimized) ===")
    
    # Focus sui casi critici
    scenarios = [
        {"name": "2. Punitivo (Beta=2)", "alpha": 1.0, "beta": 2.0, "density": 0.5},
        {"name": "7. Sparso Punitivo",   "alpha": 1.0, "beta": 2.0, "density": 0.2}, 
    ]
    
    strategies = [
        ("V4 (Regret)", init_v4_ultimate),
        ("V5 (2-OPT Repair)", init_v5_optimized)
    ]

    for sc in scenarios:
        print(f"\n---> SCENARIO: {sc['name']}")
        try:
            p = Problem(num_cities, alpha=sc['alpha'], beta=sc['beta'], density=sc['density'], seed=42)
            baseline = p.baseline()
        except: continue
            
        print(f"\n{'STRATEGIA':<20} | {'COSTO':<12} | {'GAP':<10} | {'TEMPO'}")
        print("-" * 60)
        
        for name, func in strategies:
            start = time.time()
            try:
                pop = func(p, pop_size)
                best = min([p.split(ind)[0] for ind in pop])
                gap = ((baseline - best) / baseline) * 100
                elapsed = time.time() - start
                print(f"{name:<20} | {best:<12.2f} | {gap:+.2f}%    | {elapsed:.2f}s")
            except Exception as e:
                print(f"{name}: Error {e}")

run_final_v5_test(num_cities=100, pop_size=100)

=== TEST DEFINITIVO: V4 vs V5 (Optimized) ===

---> SCENARIO: 2. Punitivo (Beta=2)

STRATEGIA            | COSTO        | GAP        | TEMPO
------------------------------------------------------------
V4 (Regret)          | 3888666.04   | +17.19%    | 1.13s
V5 (2-OPT Repair)    | 0.16         | +100.00%    | 0.98s

---> SCENARIO: 7. Sparso Punitivo

STRATEGIA            | COSTO        | GAP        | TEMPO
------------------------------------------------------------
V4 (Regret)          | 5182264.75   | +2.85%    | 1.28s
V5 (2-OPT Repair)    | 73.35        | +100.00%    | 1.10s


In [ ]:
import time
import random

strategies = [
    ("V0 (Smart)", smart_population_initialization),
    ("V1 (Super)", super_smart_initialization),
    ("V2 (C&W)", super_smart_initialization_v2),
    ("V3 (Insertion)", super_smart_initialization_v3),
    ("V4 (Regret/Gold)", init_v4_ultimate),
    ("V5 (2-Opt)", init_v5_optimized)
]

scenarios = [

    {"name": "1. Standard",         "a": 1.0, "b": 1.0, "d": 0.5},

    {"name": "2. Economy (Green)",  "a": 1.0, "b": 0.5, "d": 0.5},
    
    {"name": "3. Punitivo (Heavy)", "a": 1.0, "b": 2.0, "d": 0.5},
    
    {"name": "4. Sparso Punitivo",  "a": 1.0, "b": 2.0, "d": 0.2}, 
    {"name": "5. Denso Punitivo",   "a": 1.0, "b": 2.0, "d": 1.0},
]

def run_comprehensive_benchmark(num_cities=100, pop_size=100):
    print(f"================================================================================")
    print(f"   BENCHMARK TOTALE: CONFRONTO STRATEGIE DI INIZIALIZZAZIONE")
    print(f"   (Città: {num_cities}, Popolazione: {pop_size})")
    print(f"================================================================================\n")
    
    for sc in scenarios:
        print(f"\n---> SCENARIO: {sc['name']}")
        print(f"     Params: Beta={sc['b']}, Density={sc['d']}")
        
        try:
            p = Problem(num_cities, alpha=sc['a'], beta=sc['b'], density=sc['d'], seed=42)
            baseline = p.baseline()
        except Exception as e:
            print(f"     [!] Errore Init Problema: {e}")
            continue
            
        print(f"\n{'STRATEGIA':<20} | {'MIGLIOR COSTO':<15} | {'GAP vs BASE':<15} | {'TEMPO'}")
        print("-" * 65)
        print(f"{'Baseline':<20} | {baseline:<15.2f} | {'0.00%':<15} | {'-':<6}")
        
        for strat_name, strat_func in strategies:
            start = time.time()
            try:
                pop = strat_func(p, pop_size)
                
                best_cost = float('inf')
                for ind in pop:
                    c, _ = p.split(ind)
                    if c < best_cost: best_cost = c
                
                elapsed = time.time() - start

                gap = ((baseline - best_cost) / baseline) * 100
                gap_str = f"{gap:+.2f}%"
                
                print(f"{strat_name:<20} | {best_cost:<15.2f} | {gap_str:<15} | {elapsed:.2f}s")
                
            except Exception as e:
                print(f"{strat_name:<20} | ERROR: {e}")

run_comprehensive_benchmark(num_cities=100, pop_size=100)

   BENCHMARK TOTALE: CONFRONTO STRATEGIE DI INIZIALIZZAZIONE
   (Città: 100, Popolazione: 100)


---> SCENARIO: 1. Standard
     Params: Beta=1.0, Density=0.5

STRATEGIA            | MIGLIOR COSTO   | GAP vs BASE     | TEMPO
-----------------------------------------------------------------
Baseline             | 18811.32        | 0.00%           | -     
Logica Inizializzazione: Focus Random
V0 (Smart)           | 18807.06        | +0.02%          | 0.84s
V1 (Super)           | 18804.26        | +0.04%          | 0.87s
   [Logic V2] Beta=1.0 -> Strategia: GEOMETRICA + GRASP (Economy)
V2 (C&W)             | 18800.71        | +0.06%          | 0.86s
   [Logic V3] Beta=1.0 -> STRATEGIA: C&W + GEOMETRICHE + GRASP (Standard/Economy)
V3 (Insertion)       | 18800.71        | +0.06%          | 0.95s
V4 (Regret/Gold)     | 18800.71        | +0.06%          | 1.02s
V5 (2-Opt)           | 0.39            | +100.00%        | 0.78s

---> SCENARIO: 2. Economy (Green)
     Params: Beta=0.5, Density=0

C:\Users\feder\AppData\Local\Temp\ipykernel_1296\1450443453.py:305: RuntimeWarning: invalid value encountered in scalar power
  inc = delta + (delta * g * self._alpha)**self._beta


V3 (Insertion)       | 1236.58         | +19.48%         | 0.93s
V4 (Regret/Gold)     | 1236.58         | +19.48%         | 1.03s
V5 (2-Opt)           | 0.63            | +99.96%         | 0.83s

---> SCENARIO: 3. Punitivo (Heavy)
     Params: Beta=2.0, Density=0.5

STRATEGIA            | MIGLIOR COSTO   | GAP vs BASE     | TEMPO
-----------------------------------------------------------------
Baseline             | 4696149.78      | 0.00%           | -     
Logica Inizializzazione: Focus Random
V0 (Smart)           | 4359320.43      | +7.17%          | 0.88s
V1 (Super)           | 4308157.36      | +8.26%          | 0.91s
   [Logic V2] Beta=2.0 -> Strategia: CLARKE & WRIGHT + GRASP (Peso Punitivo)
V2 (C&W)             | 3888666.04      | +17.19%         | 0.94s
   [Logic V3] Beta=2.0 -> STRATEGIA: C&W + INSERTION + GRASP (Anti-Peso)
V3 (Insertion)       | 3888666.04      | +17.19%         | 0.99s
V4 (Regret/Gold)     | 3888666.04      | +17.19%         | 1.07s
V5 (2-Opt)           | 

In [ ]:
def init_v6_linear_boost(problem, pop_size):
    pop = []

    try: pop.append(problem.generate_clarke_wright_individual())
    except: pass
    try: pop.append(problem.generate_regret_insertion_individual())
    except: pass
    pop.append(problem.generate_nearest_neighbor_individual())

    if 0.9 <= problem.beta <= 1.1:
        cw_quota = int(pop_size * 0.5)
        for _ in range(cw_quota):
            pop.append(problem.generate_randomized_clarke_wright_individual(top_k=5))
            
    elif problem.beta >= 1.5:
        pop.append(problem.generate_gold_sorted_individual())
        for _ in range(int(pop_size * 0.3)): pop.append(problem.generate_grasp_individual(k=6))
    else:
        pop.append(problem.generate_sweep_individual())
        for _ in range(int(pop_size * 0.3)): pop.append(problem.generate_grasp_individual(k=3))

    while len(pop) < pop_size:
        l = list(range(1, len(problem._graph))); random.shuffle(l); pop.append(l)
        
    return pop

In [ ]:
def run_linear_boost_test(num_cities=100, pop_size=100):
    print(f"=== TEST FOCUS BETA=1: RANDOMIZED C&W ===")
    
    scenarios = [
        {"name": "1. Standard (Beta=1.0)", "a": 1.0, "b": 1.0, "d": 0.5},
        {"name": "5. Denso (Beta=1.0)",    "a": 1.0, "b": 1.0, "d": 1.0},
    ]
    
    strategies = [
        ("V4 (Regret)", init_v4_ultimate),       
        ("V6 (Linear Boost)", init_v6_linear_boost) 
    ]

    for sc in scenarios:
        print(f"\n---> SCENARIO: {sc['name']}")
        try:
            p = Problem(num_cities, alpha=sc['a'], beta=sc['b'], density=sc['d'], seed=42)
            baseline = p.baseline()
        except: continue
            
        print(f"\n{'STRATEGIA':<20} | {'MIGLIOR COSTO':<15} | {'GAP':<10} | {'TEMPO'}")
        print("-" * 60)
        
        for name, func in strategies:
            start = time.time()
            try:
                pop = func(p, pop_size)
                best = min([p.split(ind)[0] for ind in pop])
                gap = ((baseline - best) / baseline) * 100
                elapsed = time.time() - start
                print(f"{name:<20} | {best:<15.2f} | {gap:+.2f}%    | {elapsed:.2f}s")
            except Exception as e:
                print(f"{name}: Error {e}")

run_linear_boost_test(num_cities=100, pop_size=100)

=== TEST FOCUS BETA=1: RANDOMIZED C&W ===

---> SCENARIO: 1. Standard (Beta=1.0)

STRATEGIA            | MIGLIOR COSTO   | GAP        | TEMPO
------------------------------------------------------------
V4 (Regret)          | 18800.71        | +0.06%    | 0.92s
V6 (Linear Boost)    | 18799.83        | +0.06%    | 1.16s

---> SCENARIO: 5. Denso (Beta=1.0)

STRATEGIA            | MIGLIOR COSTO   | GAP        | TEMPO
------------------------------------------------------------
V4 (Regret)          | 18257.56        | +0.05%    | 0.75s
V6 (Linear Boost)    | 18257.34        | +0.05%    | 1.00s


In [ ]:
def init_v7_hybrid_linear(problem, pop_size):
    pop = []
    
    try: pop.append(problem.generate_clarke_wright_individual())
    except: pass
    pop.append(problem.generate_nearest_neighbor_individual())
    
    if 0.9 <= problem.beta <= 1.1:
        cw_quota = int(pop_size * 0.6) 
        for _ in range(cw_quota):
            ind = problem.generate_randomized_clarke_wright_individual(top_k=5)
            ind = problem.fast_2opt_safe(ind, max_iter=300)
            pop.append(ind)
            
    elif problem.beta >= 1.5:
        pop.append(problem.generate_gold_sorted_individual())
        try: pop.append(problem.generate_regret_insertion_individual())
        except: pass
        for _ in range(int(pop_size * 0.3)): pop.append(problem.generate_grasp_individual(k=6))
    else:
        pop.append(problem.generate_sweep_individual())
        for _ in range(int(pop_size * 0.3)): pop.append(problem.generate_grasp_individual(k=3))
    while len(pop) < pop_size:
        l = list(range(1, len(problem._graph))); random.shuffle(l); pop.append(l)
        
    return pop

In [ ]:
import time

scenarios = [
    {"name": "1. Standard",         "a": 1.0, "b": 1.0, "d": 0.5},
    {"name": "2. Economy (Beta<1)", "a": 1.0, "b": 0.5, "d": 0.5},
    {"name": "3. Punitivo (Beta=2)","a": 1.0, "b": 2.0, "d": 0.5},
    {"name": "4. Sparso Punitivo",  "a": 1.0, "b": 2.0, "d": 0.2}, 
    {"name": "5. Denso Punitivo",   "a": 1.0, "b": 2.0, "d": 1.0}, 
]

strategies = [
    ("V0 (Smart)", smart_population_initialization),
    ("V1 (Super)", super_smart_initialization),
    ("V2 (C&W)", super_smart_initialization_v2),
    ("V3 (Insert)", super_smart_initialization_v3),
    ("V4 (Regret)", init_v4_ultimate),
    ("V6 (LinBoost)", init_v6_linear_boost),
    ("V7 (Hybrid)", init_v7_hybrid_linear)
]

def run_global_benchmark(num_cities=100, pop_size=100):
    print(f"{'='*80}")
    print(f"   BENCHMARK COMPLETO: TUTTE LE STRATEGIE vs TUTTI GLI SCENARI")
    print(f"   (Città: {num_cities}, Pop: {pop_size})")
    print(f"{'='*80}\n")
    
    for sc in scenarios:
        print(f"\n---> SCENARIO: {sc['name']}")
        print(f"     Params: Beta={sc['b']}, Density={sc['d']}")
        
        try:
            p = Problem(num_cities, alpha=sc['a'], beta=sc['b'], density=sc['d'], seed=42)
            baseline = p.baseline()
        except Exception as e:
            print(f"     [!] Errore creazione problema: {e}")
            continue
            
        print(f"\n{'STRATEGIA':<20} | {'MIGLIOR COSTO':<15} | {'GAP vs BASE':<15} | {'TEMPO'}")
        print("-" * 65)
        print(f"{'Baseline':<20} | {baseline:<15.2f} | {'0.00%':<15} | {'-':<6}")
        
        for strat_name, strat_func in strategies:
            start = time.time()
            try:
                pop = strat_func(p, pop_size)
                
                best_cost = float('inf')
                for ind in pop:
                    c, _ = p.split(ind) 
                    if c < best_cost: best_cost = c
                
                elapsed = time.time() - start

                gap = ((baseline - best_cost) / baseline) * 100
                gap_str = f"{gap:+.2f}%"
                
                print(f"{strat_name:<20} | {best_cost:<15.2f} | {gap_str:<15} | {elapsed:.2f}s")
            except Exception as e:
                print(f"{strat_name:<20} | ERROR: {e}")

run_global_benchmark(num_cities=100, pop_size=100)

   BENCHMARK COMPLETO: TUTTE LE STRATEGIE vs TUTTI GLI SCENARI
   (Città: 100, Pop: 100)


---> SCENARIO: 1. Standard
     Params: Beta=1.0, Density=0.5

STRATEGIA            | MIGLIOR COSTO   | GAP vs BASE     | TEMPO
-----------------------------------------------------------------
Baseline             | 18811.32        | 0.00%           | -     
Logica Inizializzazione: Focus Random
V0 (Smart)           | 18807.06        | +0.02%          | 0.85s
V1 (Super)           | 18804.26        | +0.04%          | 0.89s
   [Logic V2] Beta=1.0 -> Strategia: GEOMETRICA + GRASP (Economy)
V2 (C&W)             | 18800.71        | +0.06%          | 0.88s
   [Logic V3] Beta=1.0 -> STRATEGIA: C&W + GEOMETRICHE + GRASP (Standard/Economy)
V3 (Insert)          | 18800.71        | +0.06%          | 0.93s
V4 (Regret)          | 18800.71        | +0.06%          | 1.06s
V6 (LinBoost)        | 18799.99        | +0.06%          | 1.37s
V7 (Hybrid)          | 18800.71        | +0.06%          | 188.30s

---> 

C:\Users\feder\AppData\Local\Temp\ipykernel_1296\3103492117.py:305: RuntimeWarning: invalid value encountered in scalar power
  inc = delta + (delta * g * self._alpha)**self._beta


V3 (Insert)          | 1236.58         | +19.48%         | 0.96s
V4 (Regret)          | 1236.58         | +19.48%         | 1.06s
V6 (LinBoost)        | 1236.58         | +19.48%         | 0.95s
V7 (Hybrid)          | 1236.58         | +19.48%         | 0.88s

---> SCENARIO: 3. Punitivo (Beta=2)
     Params: Beta=2.0, Density=0.5

STRATEGIA            | MIGLIOR COSTO   | GAP vs BASE     | TEMPO
-----------------------------------------------------------------
Baseline             | 4696149.78      | 0.00%           | -     
Logica Inizializzazione: Focus Random
V0 (Smart)           | 4399389.97      | +6.32%          | 0.89s
V1 (Super)           | 4322105.19      | +7.96%          | 0.90s
   [Logic V2] Beta=2.0 -> Strategia: CLARKE & WRIGHT + GRASP (Peso Punitivo)
V2 (C&W)             | 3888666.04      | +17.19%         | 0.89s
   [Logic V3] Beta=2.0 -> STRATEGIA: C&W + INSERTION + GRASP (Anti-Peso)
V3 (Insert)          | 3888666.04      | +17.19%         | 0.98s
V4 (Regret)          |